In [ ]:
%pip install git+https://github.com/haven-jeon/PyKoSpacing.git

In [ ]:
%pip install git+https://github.com/ssut/py-hanspell.git

In [8]:
%pip install konlpy

In [ ]:
%pip install customized_konlpy

In [ ]:
%pip install PyKomoran

## **자연어 처리**

In [41]:
import numpy as np
import pandas as pd
import itertools

import konlpy
from konlpy.tag import Okt
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from collections import Counter
from pykospacing import Spacing
from hanspell import spell_checker
import re
from PyKomoran import *
komoran = Komoran("STABLE")

okt = Okt()
spacing = Spacing()

## **okt 사용자 정의 사전 열기**

In [11]:
import os

os.chdir('/usr/local/lib/python3.7/dist-packages/konlpy/java')
os.getcwd() 
# os.makedirs('./aaa') # 새로운 사용자 정의 사전 만들기

!cp /usr/local/lib/python3.7/dist-packages/konlpy/java/open-korean-text-2.1.0.jar /usr/local/lib/python3.7/dist-packages/konlpy/java/aaa/open-korean-text-2.1.0.jar
!jar xvf /usr/local/lib/python3.7/dist-packages/konlpy/java/aaa/open-korean-text-2.1.0.jar
#/usr/local/lib/python3.7/dist-packages/konlpy/java/open-korean-text-2.1.0.jar

cp: cannot create regular file '/usr/local/lib/python3.7/dist-packages/konlpy/java/aaa/open-korean-text-2.1.0.jar': No such file or directory
java.io.FileNotFoundException: /usr/local/lib/python3.7/dist-packages/konlpy/java/aaa/open-korean-text-2.1.0.jar (No such file or directory)
	at java.base/java.io.FileInputStream.open0(Native Method)
	at java.base/java.io.FileInputStream.open(FileInputStream.java:219)
	at java.base/java.io.FileInputStream.<init>(FileInputStream.java:157)
	at java.base/java.io.FileInputStream.<init>(FileInputStream.java:112)
	at jdk.jartool/sun.tools.jar.Main.run(Main.java:407)
	at jdk.jartool/sun.tools.jar.Main.main(Main.java:1680)


In [10]:
# 사용자 사전 열기
with open(f"/usr/local/lib/python3.7/dist-packages/konlpy/java/org/openkoreantext/processor/util/noun/names.txt") as f:
    data = f.read()

FileNotFoundError: ignored

In [41]:
# 새로운 단어 추가
data += '이가탄\n인사돌\n'

In [ ]:
# 파일 새롭게 저장
with open("/usr/local/lib/python3.7/dist-packages/konlpy/java/org/openkoreantext/processor/util/noun/names.txt", 'w') as f:
    f.write(data)

!jar cvf /usr/local/lib/python3.7/dist-packages/konlpy/java/aaa/open-korean-text-2.1.0.jar * 

In [ ]:
# 확인용
a = spell_checker.check('영양제')
result = a.checked
print(result)

okt.morphs('영양제')

영양제


['영양제']

## csv 파일 불러오기 & 불용어

In [217]:
#파일 직접 업로드하기
from google.colab import files
myfile = files.upload()

Saving df_잇몸영양제_내용.csv to df_잇몸영양제_내용.csv


In [218]:
#io와 pandas 모듈 import
import io
import pandas as pd

#pd.read_csv로 csv파일 불러오기
df_잇몸영양제_내용 = pd.read_csv(io.BytesIO(myfile['df_잇몸영양제_내용.csv']))
df_잇몸영양제_내용.head()

,내용
0,양치질을 아무리 잘한다고 하더라도\n잘못된 양치질 또는 잇몸 자체가 약한 경우에는\...
1,아무리 하루 세 번 양치를 잘한다 해도\n치아에는 쉽게 문제가 생기곤 하는데...
2,나이가 드니 치아도 약해지나 봐요.\n옛날이면 돌도 씹어 먹었을 텐데\n이제는 그토...
3,잇몸 영양제 조금 아쉬웠던 이유\n한번씩 잇몸이 욱신대는 느낌이 느껴질때가 있어요....
4,어머니께서 예전부터 잇몸이 좋지 않아\n최근에 임플란트를 예정 중에 계신데 치과에서...


In [219]:
# 불용어 리스트 만들기
# bul_word = []
stopwords = ['르', '과', '까지', '곳', '다시금', '제현', '귀엽다', '과', '게', '때','쿠','쵸','존',
             '링','보다','경우','반면','되다','있다','자나','살다','혹시','경우','때문','정도','사항',
                   '해당','겁니다','이것','저것','그것','돋움','신명', '태명', '이미', '돋움',
                   '동안','거기','저기','여기','대부분','누구','무엇','고딕','만큼','굴림','감사','건지','텐데',
                   '안녕','그렇다','이번','걸로','수고','겁니까','그간','그건','그때','글쓴이','누가','니다','다면',
                   '뭔가','또한','하다','이다','되다','다능','자다','자체','서체','지나가다','잇다','같다','미치','있다','없다', '여지','관련',
                   '생각', '현재', '진행', '사람', '마음', '남산', '내용', '현실','음','막','전하다','들다','열다',
                   '두다','서다','산하','지금','주변','대상','부분','요즘','하루','마련','시간','이상','행위',
                   '이하','바로 가기','바로가기','제가','먹다','보이다','지나다','콜론','각','대하다','따르다','인하다','전','후','예','한','다','보다','그렇다','좋다',
                   '나다','너무','정말','내다','많다','그리다','물론','조금','다시','느끼다','없이','특히','여러','다른','이런','가지다','어떻다','통하다', '위하다'
                   '다니다','나서다','들다','열다','어떻게','막상','가다','오다','말다','계속','쉬다','혜량',
                   '무현','여쭈다','베타','되묻다','뜨다', '다녀오다','서녀','붙다','찾다','생각나다','앉다','바니','이르다','크다',
                   '그냥','그저','바로','시키다','허다','내내','해량','걸다', '이렇다','이렇게','켜다','어린이날','그날','남주','제일','지금껏',
             '진짜','끌다','받다','넣다','무지','태주','아무튼','알다','틀다','마치','부리','익히다','만들다','잡다','아주','영재','대구','서울','매우','나오다',
             '뫄다','케이','히나','이치','경허','비다','찌다','받다','나가다','일어나다','오키','일단','우왕','드리다',
             '대다','덮다','바잔','스다','들어가다','놓아두다','놓다','틀다','들다','사다','팔다','이대로','서교동','조지','치다','그다음','다음',
             '마티','파주시','태평','잘나다','모르다','알다','알고있다','주전','작대기','아이스','들뜨다','푸다','아우','쿠쿠','가장','몰다','뭐하다', '눕다', '누르다'
             '존나','데리다','트다','쓰다','타임','라인','벌리다','타다','이제','위하다','다니다','한동안','저번','치과','마이','어쩌다','사실','그렇게','이렇게',
             '해보다','해저','가세','시마','블루','부르다','야지','에서','의']
                   
# 기존 불용어 리스트에 추가
# stopwords.append('홈페이지')
stopwords_set = set(stopwords)

# for word_tokens in kospacing_list:
#     bul_word.append([word for word in okt.morphs(word_tokens) if not word in stopwords_set])

# # print(bul_word)

# # 이중리스트 제거
# blist = sum(bul_word, [])
# print(blist)

In [220]:
##정규화
def normalizing(context):
    return okt.normalize(context)

def extract_word(context):
    hangul = re.compile('[^가-힣]')
    result = hangul.sub(' ', context)
    result = result.replace(' ','')
    return result

##띄어쓰기
#1 pykospacing

def spacing_text1(context):
    try:
      return spacing(context)
    except:
      return context

#2 pyhanspell
def spacing_text2(context):
    try:
      a = spell_checker.check(context)
      result = a.checked
      return result
    except:
      return context

def text_tokenizer(context):
    return [w.get_first() for w in komoran.get_list(context)]

def noun_extractor(context):
    return [w.get_first() for w in komoran.get_list(context) if w.get_second() in ['NNP','NNG']]

# def text_nv(context):
#     all = context.pos(context, join = True)
#     return [word for word in all if word in ['Noun','Adjective','Verb']]

# def text_nv(context):
#     all = okt.pos(context, join = True)
#     return [re.sub('[^가-힣]','',word) for word in all if ('Noun' in word or 'Verb' in word or 'Adjective' in word)]

def text_nv(context):
       return [w.get_first()+'다' if w.get_second() in ['VV','VA'] else w.get_first() for w in komoran.get_list(context) if w.get_second() in ['NNP','NNG','VA','VV','MAG']]

In [221]:
def context_nlp(df):
    title = input('파일 내용: ')
    #df = df.drop('Unnamed: 0', axis=1)
    df = df.dropna(axis=0)
    print('시작')

    df['내용'] = df['내용'].apply(lambda x:normalizing(x))
    df['내용'] = df['내용'].apply(lambda x:extract_word(x))
    df = df.dropna(axis=0)
    print('정규화 완')
    
    df['내용'] = df['내용'].apply(lambda x:spacing_text1(x))
    df = df.dropna(axis=0)
    print('띄1 완')

    # df['내용'] = df['내용'].apply(lambda x:spacing_text2(x))
    # df = df.dropna(axis=0)
    # print('띄2 완')

    df['all_tokens'] = df['내용'].apply(lambda x:text_tokenizer(x))
    df = df.dropna(axis=0)
    print('토크나이징 완')
    
    df['tokens'] = df['내용'].apply(lambda x:text_nv(x))
    df['nouns'] = df['내용'].apply(lambda x:noun_extractor(x))
    print('토큰 추출 완')

    # 한글자 형태소 제거
    df['tokens'] = df['tokens'].apply(lambda x:[w for w in x if len(w) > 1])
    df['all_tokens'] = df['all_tokens'].apply(lambda x:[w for w in x if len(w) > 1])
    df['nouns'] = df['nouns'].apply(lambda x:[w for w in x if len(w) > 1])

    # 불용어 제거
    df['tokens'] = df['tokens'].apply(lambda x:[w for w in x if not w in stopwords_set])
    df['nouns'] = df['nouns'].apply(lambda x:[w for w in x if not w in stopwords_set])
    #df['all_tokens'] = df['all_tokens'].apply(lambda x:[w for w in x if not w in stopwords_set])

    df.to_csv('/content/{0}_완료.csv'.format(title),encoding='utf-8-sig')

    return df



In [222]:
aqw = context_nlp(df_잇몸영양제_내용)

파일 내용: 잇몸영양제_내용
시작
정규화 완
띄1 완
토크나이징 완
토큰 추출 완


In [223]:
aqw

,내용,all_tokens,tokens,nouns
0,양치질을 아무리 잘한다고 하더라도 잘못된 양치질 또는 잇몸 자체가 약한 경우에는 잇...,"[양치질, 아무리, ㄴ다고, 더라도, 잘못, 양치질, 또는, 잇몸, 자체, 약하, ...","[양치질, 아무리, 잘못, 양치질, 또는, 잇몸, 약하다, 잇몸, 질환, 피하다, ...","[양치질, 잘못, 양치질, 잇몸, 잇몸, 질환, 이야기, 음식, 방문, 관리, 양치..."
1,아무리 하루 세 번 양치를 잘한다 해도 치아에는 쉽게 문제가 생기곤 하는데 얼마 전...,"[아무리, 하루, 양치, ㄴ다, 아도, 치아, 문제, 생기, 는데, 얼마, 부터, ...","[아무리, 양치, 치아, 쉽다, 문제, 생기다, 얼마, 칫솔, 출혈, 함께, 통증,...","[양치, 치아, 문제, 얼마, 칫솔, 출혈, 통증, 시작, 잘못, 양치질, 치아, ..."
2,나이가 드니 치아도 약해지나 봐요옛날이면 돌도 씹어 먹었을 텐데 이제는 그토록 좋아...,"[나이, 아도, 옛날, 이제, 그토록, 좋아하, 오징어, 하나, 어요, 모르, 다가...","[나이, 옛날, 씹다, 그토록, 좋아하다, 오징어, 씹다, 통증, 확실히, 잇몸, ...","[나이, 옛날, 오징어, 통증, 잇몸, 나이, 전반, 신체, 노화, 일상생활, 불편..."
3,잇몸 영양제조금 아쉬웠던 이유 한번씩 잇몸이 욱신대는 느낌이 느껴질 때가 있어요 그...,"[잇몸, 영양제, 조금, 아쉽, 이유, 한번, 잇몸, 다는, 느낌, 느껴지, 어요,...","[잇몸, 영양제, 아쉽다, 이유, 한번, 잇몸, 신다, 느낌, 느껴지다, 느낌, 생...","[잇몸, 영양제, 이유, 한번, 잇몸, 느낌, 느낌, 찬물, 느낌, 잇몸, 영양제,..."
4,어머니께서 예전부터 잇몸이 좋지 않아 최근에 임플란트를 예정 중에 계신데 치과에서 ...,"[어머니, 께서, 예전, 부터, 잇몸, 최근, 예정, 계시, ㄴ데, 치과, 에서, ...","[어머니, 예전, 잇몸, 최근, 예정, 계시다, 상태, 치료, 불혹, 바라보다, 이...","[어머니, 예전, 잇몸, 최근, 예정, 상태, 치료, 불혹, 충치, 평생, 치아, ..."
...,...,...,...,...
95,안녕하세욤 치아는 어릴 때부터 꾸준하게 관리를 해주어야 해요평생 쓰는 치아이기 때문...,"[안녕하세요, 어리, 부터, 꾸준, 관리, 어야, 평생, 치아, 때문, 관리, 소홀...","[어리다, 관리, 평생, 치아, 관리, 소홀, 건강, 빨간불, 켜지다, 금액, 되돌...","[관리, 평생, 치아, 관리, 소홀, 건강, 빨간불, 금액, 상태, 치아, 잇몸, ..."
96,잇몸 부엇을 때 잇몸 내려 앉음 걱정 잇몸 영양제를 먹어보자 지난 년간의 치아교정으...,"[잇몸, 잇몸, 내리, 걱정, 잇몸, 영양제, 지나, 년간, 치아, 교정, 으로, ...","[잇몸, 붓다, 잇몸, 내리다, 걱정, 잇몸, 영양제, 치아, 교정, 교정, 확실히...","[잇몸, 잇몸, 걱정, 잇몸, 영양제, 치아, 교정, 교정, 장치, 잇몸, 데미지,..."
97,부광약품 웰치부광약품부 광약품 웰치잇몸영양제 치아 영양제 치아 건강 잇몸건강 잇몸건...,"[부광약품, 웰치, 부광약품, 약품, 웰치, 잇몸, 영양제, 영양제, 건강, 잇몸,...","[부광약품, 웰치, 부광약품, 약품, 웰치, 잇몸, 영양제, 영양제, 건강, 잇몸,...","[부광약품, 웰치, 부광약품, 약품, 웰치, 잇몸, 영양제, 영양제, 건강, 잇몸,..."
98,이웃님들 잇몸영양제 챙기고 있으신가요 저는 치아교정을 결혼하고 했는데 아무래도 늦은...,"[이웃, 잇몸, 영양제, 챙기, 으시, ㄴ가요, 치아, 교정, 결혼, 는데, 아무래...","[이웃, 잇몸, 영양제, 챙기다, 치아, 교정, 결혼, 아무래도, 늦다, 나이, 치...","[이웃, 잇몸, 영양제, 치아, 교정, 결혼, 나이, 치아, 교정, 시작, 잇몸, ..."


### 단어 카운트 


In [224]:
# a = list(itertools.chain(*den_wan['tokens']))
den_wan = pd.read_csv('/content/잇몸영양제_내용_완료.csv')
li = den_wan['tokens']
# # 이중리스트 제거
# 리스트가 문자열로 저장된 경우 (ex> ["['dd','aa']"])
import ast
n_list = []

for i in li:
  i = ast.literal_eval(i)
  n_list.append(i)

# 이중리스트 제거
a = sum(n_list, [])

In [225]:
print(a)

['양치질', '아무리', '잘못', '양치질', '또는', '잇몸', '약하다', '잇몸', '질환', '피하다', '이야기', '특히나', '음식', '많이', '방문', '관리', '양치', '잇몸', '양치질', '잘못', '잇몸', '약하다', '잇몸', '관리', '함께', '보통', '올바르다', '칫솔', '가르치다', '잇몸', '관리', '방법', '여럿', '가르치다', '우선', '잇몸', '관리', '잇몸', '마사지', '아기', '양치질', '어렵다', '유치', '거즈', '이용', '잇몸', '마사지', '방식', '칫솔', '이용', '잇몸', '마사지', '싣다', '사용', '방법', '싣다', '사용', '방법', '올바르다', '양치질', '함께', '잇몸', '관리', '잇몸', '영양제', '함께', '섭취', '관리', '해주', '추천', '받았다', '주기', '검진', '필수', '상태', '검진', '안내', '돌아오다', '생활', '습관', '돌아보다', '잇몸', '관리', '영양제', '알아보다', '시작', '보통', '잇몸', '알려지다', '대표', '영양제', '제품', '성분', '전문', '브랜드', '위주', '검색', '잇몸', '알리다', '프로', '폴리스', '성분', '위주', '알아보다', '프로', '폴리스', '일반', '프로', '폴리스', '다르다', '나무', '추출', '성분', '수액', '이용', '생산', '초록색', '띄다', '프로', '폴리스', '이름', '붙이다', '일반', '프로', '폴리스', '잇몸', '성분', '성분', '중점', '알아보다', '맛있다', '헬퍼', '영양제', '고요', '이름', '잇몸', '영양제', '원료', '알아보다', '프로', '폴리스', '특징', '살펴보다', '일반', '원료', '브라질', '정부', '공식', '인증', '원료', '사용', '브라질', '정부', '원료', '성분', '제조', '공정'

In [226]:
# 리스트를 딕셔너리로 생성하고 빈도수에 따라 키워드를 정렬하는 함수
def count_f(my_list):
    count = {} # 딕셔너리 생성

    for item in my_list:
        count[item] = count.get(item, 0) + 1
    
    final_df = (pd.DataFrame(list(count.items()),
                            columns=['keyword', 'count']))
    
    # 빈도수가 10보다 작다면 해당 행 삭제
    d_row = final_df[final_df['count'] < 5].index
    final_df = final_df.drop(d_row)

    # count 칼럼에 따라서 내림차순 정렬
    final_df = final_df.sort_values(by=['count', 'keyword'], ascending=False)
    final_df = final_df.reset_index(drop=True) # index 초기화    
    
    return final_df

# 데이터프레임 csv파일로 저장하는 함수
# def df_export_csv(my_df):
#     my_file = input('저장할 파일의 이름을 입력하세요 : ')
#     exp = my_df.to_csv(('{0}.csv').format(my_file), index=False, header=False, encoding='cp949')
    
#     return exp

a2 = count_f(a)
print(a2)
# b = df_export_csv(a2)
a2.to_csv('/content/잇몸영양제_내용_count.csv',encoding='utf-8-sig')

    keyword  count
0        잇몸   1553
1       영양제    918
2        건강    698
3       챙기다    370
4        성분    333
..      ...    ...
782      검사      5
783    건강식품      5
784     강하다      5
785      강추      5
786      감기      5

[787 rows x 2 columns]


In [26]:
# 한글과 공백 제외 모두 제거 : 띄어쓰기 유지, 구두점 제거

def gonggack_remove(df):
  df_replace = df.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]',"")
  return df_replace

ab_df = gonggack_remove(df_치과_제목['제목'])
ab_df[:10]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.


0           치과전용칫솔 치석 제거 잘 되는 칫솔 추천
1         치과 공포증 치료 받기 무서워요 극복에 중요한
2           아주대근처 매탄동 우리사랑치과에서 치료시작
3           오즈어린이치과의원 동래점만세아 신경치료ㅠㅠ
4          치과치료스케일링잇몸질환  라이나생명치아보험 
5              김해치과 진료비 비쌀 이유가 있을까요
6               순천치과에서 앞니 임플란트 했어요 
7    잇몸이 부었을때는 치과치료와 인사돌플러스 함께 병행하기
8                      인천교정치과 시기까지도
9                     창원치과 걱정 그만할게요
Name: 제목, dtype: object

In [44]:
# 데이터프레임 특정 열값을 리스트로 변환
spacing_cen = []
for sc in ab_df:
    spacing_cen.append(sc)
# print(spacing_cen)

# 리스트 문자열 변환
result = ' '.join(s for s in spacing_cen)
# print(result)

In [45]:
# 띄어쓰기가 없는 문장 임의로 만들기
new_sent = result.replace(" ", '') 
# print(new_sent)

In [46]:
# 띄어쓰기
spacing = Spacing()
kospacing_sent = spacing(result) 
# print(sent) # 원본 확인용   
# print(kospacing_list)

In [47]:
# 문법 정규화
kospacing_list = [okt.normalize(kospacing_sent)]

In [49]:
# 명사 추출

n_list=[]
for word in blist:
    n_list.append(okt.nouns(word))

# 이중리스트 제거
final_n = sum(n_list, [])
print(final_n)

['전용', '칫솔', '치석', '제거', '칫솔', '추천', '공포증', '치료', '극복', '아주대', '근처', '매탄동', '우리', '사랑', '치료', '시작', '오즈', '어린이', '의원', '동래', '점', '만', '세아', '신경치료', '치료', '스케일링', '잇몸', '질환', '라이나', '생명', '치아', '보험', '김해', '진료', '비', '이유', '순', '천', '앞니', '임플란트', '잇몸', '이', '치료', '인', '사', '돌', '플러스', '병행', '인천', '교', '정치', '시기', '창원', '걱정', '의정부', '비용', '부담', '서초동', '추천', '마음가짐', '일산', '동물병원', '알콩', '이', '첫', '진료', '관악구', '교정', '거꾸로', '반대', '교합', '비', '수술', '교정', '치료', '로', '방배동', '신경치료', '실패', '로', '발치', '임플란트', '케이스', '수원', '임플란트', '선택', '기준', '가지', '턱관절', '어디', '로', '가야', '고민', '절개', '고', '법', '은', '잇몸', '성형', '노하우', '서면', '임플란트', '를', '찾기', '상태', '마케팅', '코', '시국', '문의', '폭주', '방법', '오산', '치', '충치', '이빨', '부평', '교', '정치', '인상', '통해', '신경치료', '대한', '정부', '역', '걱정', '이', '비용', '사랑니', '공장', '잎사귀', '방문', '기사', '설', '주의', '교대역', '후회', '영등포', '더', '이', '유', '웹툰', '웃음', '가스', '수면', '치료', '어린이', '범어역', '치아', '문제', '심화', '대체', '진료', '로', '가산동', '은', '어디', '광화문역', '디지털', '임플란트', '수술', '혜택', '부산', '수술', '결과',

In [ ]:
a = list(itertools.chain(*den['tokens']))
b = count_f(a)

# # 이중리스트 제거
# final_n(변수) = sum(n_list(리스트), [])
# print(final_n)